In [285]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from scipy import stats

In [299]:
tdata = pd.read_csv('train.csv',index_col=0)

In [300]:
tdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [302]:
names= tdata.Name.values

In [303]:
extract = {'Mr.':1,'Mrs.':2,'Miss.':3,'Master.':4,'Rev.':5,'Dr.':6,'Mrs':7,'Col.':8,'Don.':9,'Contess.':10,'Mlle.':11,'Mme.':12}
# a mapping dictionary
mapName =[0]*len(tdata)
for i,item in enumerate(names):
    parse = str(item).split(' ')
    for parts in parse:
        if parts in extract.keys(): mapName[i]=extract[parts]

In [304]:
# replace the 'Name' column 
tdata['Name']=mapName

In [305]:
tdata.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,1,male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,2,female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,2,female,35.0,1,0,113803,53.1000,C123,S
5,0,3,1,male,35.0,0,0,373450,8.0500,NaN,S


In [306]:
# fill in the missing Age

# calculate mode of age based on 3 different cabin classes
sub_dt = tdata.loc[tdata.Pclass==1]
a = np.array(sub_dt[sub_dt.Age.notnull()].Age)
modc1 = stats.mode(a)[0][0]
sub_dt = tdata.loc[tdata.Pclass==2]
a = np.array(sub_dt[sub_dt.Age.notnull()].Age)
modc2 = stats.mode(a)[0][0]
sub_dt = tdata.loc[tdata.Pclass==3]
a = np.array(sub_dt[sub_dt.Age.notnull()].Age)
modc3 = stats.mode(a)[0][0]
#  create a mapping dictionary
mapAge = {1:modc1,2:modc2, 3:modc3}

In [307]:
tdata['Age']=tdata['Age'].fillna(tdata['Pclass'].map(mapAge)) # fill in mode number based on cabin class  to the training data

In [308]:
# separate the character and number in the Cabin column
extCabin = tdata.Cabin.values

In [309]:
extCabinTable = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'F G':8,'F E':9}
mapCabin=[0]*len(tdata)

In [310]:
for i,item in enumerate(tdata.Cabin.values):
    if item is not np.nan:
        parse = re.split('(\d+)',str(item))
        for parts in parse:
            if parts in extCabinTable.keys(): mapCabin[i]=extCabinTable[parts]

In [311]:
tdata['CabinC']=mapCabin

In [312]:
mapCabinN=[0]*len(tdata)
for i,item in enumerate(tdata.Cabin.values):
    if item is not np.nan:
        parse = re.split('(\d+)',str(item))
        if len(parse)==1: continue  # deal with the case that some rows missing Cabin number but with cabin char
        tmp = [int(x) for x in parse[1:2]]
        mapCabinN[i]=np.mean(tmp)

In [313]:
tdata['CabinN']=mapCabinN

In [314]:
tdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 13 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
CabinC      891 non-null int64
CabinN      891 non-null float64
dtypes: float64(3), int64(6), object(4)
memory usage: 97.5+ KB


In [315]:
# deal with missing values in Embarked
tdata.loc[tdata.Embarked.isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,CabinC,CabinN
PassengerId,,,,,,,,,,,,,
62,1,1,3,female,38.0,0,0,113572,80.0,B28,NaN,2,28.0
830,1,1,2,female,62.0,0,0,113572,80.0,B28,NaN,2,28.0


#### Because the harbor which boarded most people is Southampton, I will assume these two records came from 'S' for simplicity (see data exploration)

In [316]:
tdata['Embarked']=tdata.Embarked.fillna(value='S')

In [317]:
tdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 13 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    891 non-null object
CabinC      891 non-null int64
CabinN      891 non-null float64
dtypes: float64(3), int64(6), object(4)
memory usage: 97.5+ KB


In [318]:
# convert column 'Sex' and 'Embarked'

# make a simple function that can take in a dictionary and convert the entire column values
# This function can be used over and over
def encod_catcoldf(df,dic:dict,col:str):
    """df: pandas dataframe; dic: dictionary that does the mapping for all possible values from desired column(1 column)"""
    """col: string format of the column name"""
    ext = df[col].values
    mapcol = [0]*len(df)
    for i,item in enumerate(ext):
        mapcol[i]=dic[item]
    
    df[col]=mapcol
    return df

In [319]:
mapSex = {'female':0,'male':1}
encod_catcoldf(tdata,mapSex,'Sex')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,CabinC,CabinN
PassengerId,,,,,,,,,,,,,
1,0,3,1,1,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0.0
2,1,1,2,0,38.0,1,0,PC 17599,71.2833,C85,C,3,85.0
3,1,3,3,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0.0
4,1,1,2,0,35.0,1,0,113803,53.1000,C123,S,3,123.0
5,0,3,1,1,35.0,0,0,373450,8.0500,NaN,S,0,0.0
6,0,3,1,1,22.0,0,0,330877,8.4583,NaN,Q,0,0.0
7,0,1,1,1,54.0,0,0,17463,51.8625,E46,S,5,46.0
8,0,3,4,1,2.0,3,1,349909,21.0750,NaN,S,0,0.0
9,1,3,2,0,27.0,0,2,347742,11.1333,NaN,S,0,0.0


In [320]:
mapEmb = {'C':0,'S':1,'Q':2}
encod_catcoldf(tdata,mapEmb,'Embarked')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,CabinC,CabinN
PassengerId,,,,,,,,,,,,,
1,0,3,1,1,22.0,1,0,A/5 21171,7.2500,NaN,1,0,0.0
2,1,1,2,0,38.0,1,0,PC 17599,71.2833,C85,0,3,85.0
3,1,3,3,0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,0.0
4,1,1,2,0,35.0,1,0,113803,53.1000,C123,1,3,123.0
5,0,3,1,1,35.0,0,0,373450,8.0500,NaN,1,0,0.0
6,0,3,1,1,22.0,0,0,330877,8.4583,NaN,2,0,0.0
7,0,1,1,1,54.0,0,0,17463,51.8625,E46,1,5,46.0
8,0,3,4,1,2.0,3,1,349909,21.0750,NaN,1,0,0.0
9,1,3,2,0,27.0,0,2,347742,11.1333,NaN,1,0,0.0


In [328]:
# remove the column 'Ticket' and 'Cabin'(which is already separted into CabinC and CabinN)
tdata=tdata.drop(columns=['Ticket','Cabin'], axis=1)

In [330]:
# save out the cleaned data
tdata.to_csv('cleaned_train.csv',index=False)

In [329]:
tdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null int64
CabinC      891 non-null int64
CabinN      891 non-null float64
dtypes: float64(3), int64(8)
memory usage: 83.5 KB
